# Welcome to the 210Pb age model script v2.3.2!

### <div style="text-align: right"> Last modified by A.A. Lehrmann 24 November 2025 </div>


### The script below will extract radioisotope data from Canberra PDFs, run the age model (from the Wellner Lab Group excel model (Appleby, 2001; Boldt et al., 2013), and plot the age model



## First, extract radioisotope data from Canberra PDFs
Run cell (press triangle that says run) and follow instructions.

In [ ]:
#Cell 1: import required libraries
import os
import pandas as pd
from PyPDF2 import PdfReader
from pathlib import Path
from datetime import datetime
import re

In [ ]:
# Cell 2: --- Helper: prefer highest version (_v2, _v3, …) for BOTH Canberra and PtSrc files ---
import os, re, numpy as np, pandas as pd

# Capture depth AND optional version (e.g., 21-24, 21-24cm, 21-24_v2, 21-24cm_v3, etc.)
DEPTH_RE = re.compile(r'(\d{1,3}-\d{1,3})(?:cm)?(?:_v(\d+))?(?=\.|_|$)', re.IGNORECASE)

def _depth_and_version(s: str):
    base = os.path.basename(str(s))
    m = DEPTH_RE.search(base)
    if not m:
        return None, 0
    depth = m.group(1)                 # standardized like "21-24"
    version = int(m.group(2)) if m.group(2) else 0
    return depth, version

def reshape_canberra_with_ptsrc_mixed(df_in: pd.DataFrame,
                                      write_to: str | None = None,
                                      ptsrc_cols: tuple[str,str] = ("Pb-210", "Pb-210 error")) -> pd.DataFrame:
    """
    Works on a single mixed table (Canberra + PtSrc rows).
    - For any duplicate depths on either side, keeps the **highest _vN**.
    - Adds H/I/J = ['ptsrc_pb210','ptsrc_pb210 error','file ptsrc'] right after 'Pb-214 error'.
    - Preserves your original header names (no renaming of 'error' columns).
    """
    if "File" not in df_in.columns:
        raise KeyError("Input dataframe must contain a 'File' column.")

    df = df_in.copy()

    # Split rows by prefix
    is_ptsrc = df["File"].astype(str).str.startswith("PtSrc_")
    can_df = df.loc[~is_ptsrc].copy()
    pt_df  = df.loc[ is_ptsrc].copy()

    # Extract (depth, version) for both sides
    can_df[["__depth__","__ver__"]] = can_df["File"].apply(lambda s: pd.Series(_depth_and_version(s)))
    pt_df[["__depth__","__ver__"]]  = pt_df["File"].apply(lambda s: pd.Series(_depth_and_version(s)))

    # **Prefer highest version for Canberra** when duplicates share the same depth
    can_df = can_df.sort_values("__ver__").drop_duplicates(subset="__depth__", keep="last")

    # Pick value/error columns in PtSrc rows
    val_col, err_col = ptsrc_cols
    if val_col not in pt_df.columns or err_col not in pt_df.columns:
        # Fallback: first two numeric columns
        num_cols = [c for c in pt_df.columns if c != "File" and np.issubdtype(pt_df[c].dtype, np.number)]
        if len(num_cols) < 2:
            for c in pt_df.columns:
                if c != "File":
                    pt_df[c] = pd.to_numeric(pt_df[c], errors="coerce")
            num_cols = [c for c in pt_df.columns if c != "File" and np.issubdtype(pt_df[c].dtype, np.number)]
        val_col, err_col = num_cols[:2]

    # Reduce PtSrc to needed columns and **prefer highest version per depth**
    q = pt_df.loc[:, ["__depth__", "__ver__", "File", val_col, err_col]].copy()
    q.rename(columns={
        "File": "file ptsrc",
        val_col: "ptsrc_pb210",
        err_col: "ptsrc_pb210 error"
    }, inplace=True)
    q["file ptsrc"] = q["file ptsrc"].map(lambda x: os.path.basename(str(x)))
    q = q.sort_values("__ver__").drop_duplicates(subset="__depth__", keep="last")

    # Merge and clean
    out = can_df.merge(q.drop(columns="__ver__"), on="__depth__", how="left")
    out.drop(columns=["__depth__","__ver__"], errors="ignore", inplace=True)

    # Desired column order (keep original header names)
    base = ["File","Pb-210","Pb-210 error","Bi-214","Bi-214 error","Pb-214","Pb-214 error"]
    hij  = ["ptsrc_pb210","ptsrc_pb210 error","file ptsrc"]
    others = [col for col in out.columns if col not in base + hij]
    out = out[[c for c in base if c in out.columns] + hij + others]

    if write_to:
        out.to_csv(write_to, index=False)
    return out


In [ ]:
#All Inputs
#Cell 3: add age model information
SCRIPT_VERSION = "v2.3.2"
OPERATOR_NAME = input("Operator name: ").strip()
_default = datetime.today().strftime("%Y%m%d")
_run = input(f"Run date [YYYYMMDD] (Enter for { _default }): ").strip() or _default
RUN_DATE = _run
CORE_NAME_RAW = input("Core name (e.g., MB1901): ").strip()

#Cell 4: session metadata
CORE_NAME = re.sub(r"[^A-Za-z0-9]+", "", CORE_NAME_RAW)
SUFFIX = f"_{CORE_NAME}_{RUN_DATE}"
META = {"operator": OPERATOR_NAME, "date": RUN_DATE, "core": CORE_NAME, "version": SCRIPT_VERSION, "suffix": SUFFIX}

#Cell 12: User inputs
folder_path = input("Enter the folder path of Canberra PDFs: ").strip()
def compose_output_name(basename: str, ext: str, directory: Path | str = Path.cwd()) -> Path:
    """
    Returns a full Path like <directory>/<basename>_<CoreName_YYYYMMDD><ext>
    """
    directory = Path(directory)
    base = f"{basename}{META['suffix']}"
    if not ext.startswith("."):
        ext = "." + ext
    return directory / f"{base}{ext}"
# Auto-generate CSV output name based on metadata
output_csv_path = compose_output_name("CanberraData", ".csv", Path(folder_path))
print(f"CSV will be saved as -> {output_csv_path}")

#Cell 14
# Prompt user for file paths

#Cell 14: Auto-search for weights CSV file
# Search for weights file in the Canberra PDFs folder
weights_pattern = ['*_weights.csv', '*-weights.csv']
weights_files = []
for pattern in weights_pattern:
    weights_files.extend(Path(folder_path).glob(pattern))

if weights_files:
    csv1_path = str(weights_files[0])  # Use the first match
    print(f"Found weights file: {csv1_path}")
else:
    # Fallback to manual input if no file found
    print("Warning: No weights file found matching pattern '*_weights.csv' or '*-weights.csv'")
    csv1_path = input("Enter the path to the sample weight CSV file manually: ").strip()
csv2_path = output_csv_path

#Cell 16: Prompt the user for the year of core
year_of_core = int(input("Enter the year of core (e.g., 2023): "))

#Cell25: Ask for depths to label "calendar years pre year of core"
depths_to_label_input = input("Enter the depths (comma-separated) where 'calendar years pre year of core' should be labeled (or type 'all' to label all intervals): ")

#Cell 26: Ask if any intervals have undetectable radioisotope amounts
missing_data_input = input("Are there any intervals with undetectable amounts of radioisotopes? (yes/no): ").strip().lower()

if missing_data_input == 'yes':
    missing_depths_input = input("Enter the depths (comma-separated) with undetectable radioisotopes: ")
    missing_depths = [float(depth.strip()) for depth in missing_depths_input.split(",")]
else:
    missing_depths = []

#Collect README notes for outputs
readme_canberra = input("Notes for Canberra data CSV (optional): ").strip()
readme_agemodel = input("Notes for Age Model CSV (optional): ").strip()
readme_plot = input("Notes for Age Model Plot PDF (optional): ").strip()


In [ ]:
#Cell 5: write readme files
def write_readme(for_path: Path, notes: str = ""):
    """
    Creates: <basename>_README_<YYYYMMDD>.txt next to the file.
    Uses pre-collected notes.
    """
    base_no_ext = for_path.with_suffix("").name
    readme_name = f"{base_no_ext}_README_{META['date']}.txt"
    readme_path = for_path.parent / readme_name
    content = [
        f"Output file: {for_path.name}",
        f"Operator: {META['operator']}",
        f"Run date (YYYYMMDD): {META['date']}",
        f"Script version: {META['version']}",
        f"Core name: {META['core']}",
        f"Notes: {notes if notes else '(none)'}",
        ""
    ]
    readme_path.write_text("\n".join(content), encoding="utf-8")
    print(f"README created -> {readme_path}")

In [ ]:
#Cell 6: Process pt_src files (UPDATED to check pages 3 and 4)
def process_ptsrc_pdf(file_path, filename):
    try:
        reader = PdfReader(file_path)
        if len(reader.pages) < 3:
            print(f"PDF file '{filename}' has less than 3 pages. Skipping.")
            return None
        
        # Try page 3 first (index 2), then page 4 (index 3)
        pages_to_check = [2]  # Start with page 3
        if len(reader.pages) >= 4:
            pages_to_check.append(3)  # Add page 4 if it exists
        
        for page_idx in pages_to_check:
            page = reader.pages[page_idx]
            text = page.extract_text()
            lines = text.split('\n')
            for line in lines:
                if 'Pb-210' in line:
                    ptsrc_pb210, PtSrc_Pb210error = line.split()[-2:]
                    print(f"Found Pb-210 data in '{filename}' on page {page_idx + 1}")
                    return {
                        'File': filename,
                        'Pb-210': float(ptsrc_pb210),
                        'Pb-210 error': float(PtSrc_Pb210error)
                    }
        
        print(f"Pb-210 not found in '{filename}' on pages 3 or 4. Skipping.")
        return None
    except Exception as e:
        print(f"Error processing PDF file '{filename}': {e}")
        return None



In [ ]:
#Cell 7: Process regular files (UPDATED to check pages 3 and 4)
def process_regular_pdf(file_path, filename):
    pb210 = pb210error = Bi214 = Bi214error = Pb214 = Pb214error = None
    try:
        reader = PdfReader(file_path)
        if len(reader.pages) < 3:
            print(f"PDF file '{filename}' has less than 3 pages. Skipping.")
            return None
        
        # Try page 3 first (index 2), then page 4 (index 3)
        pages_to_check = [2]  # Start with page 3
        if len(reader.pages) >= 4:
            pages_to_check.append(3)  # Add page 4 if it exists
        
        data_found = False
        for page_idx in pages_to_check:
            page = reader.pages[page_idx]
            text = page.extract_text()
            lines = text.split('\n')
            for line in lines:
                if 'Pb-210' in line:
                    pb210, pb210error = line.split()[-2:]
                elif 'Bi-214' in line:
                    Bi214, Bi214error = line.split()[-2:]
                elif 'Pb-214' in line:
                    Pb214, Pb214error = line.split()[-2:]
            
            # Check if we found all required data
            if pb210 is not None and Bi214 is not None:
                data_found = True
                print(f"Found nuclide data in '{filename}' on page {page_idx + 1}")
                break
        
        if not data_found:
            if pb210 is None or pb210error is None:
                print(f"Pb-210 not found in '{filename}' on pages 3 or 4. Skipping.")
                return None
            if Bi214 is None or Bi214error is None:
                print(f"Bi-214 not found in '{filename}' on pages 3 or 4. Skipping.")
                return None
        
        if Pb214 is None or Pb214error is None:
            print(f"Pb-214 not found in '{filename}'. Using NaN values.")
            Pb214 = Pb214error = 'NaN'
        
        return {
            'File': filename,
            'Pb-210': float(pb210),
            'Pb-210 error': float(pb210error),
            'Bi-214': float(Bi214),
            'Bi-214 error': float(Bi214error),
            'Pb-214': float(Pb214) if Pb214 != 'NaN' else float('nan'),
            'Pb-214 error': float(Pb214error) if Pb214error != 'NaN' else float('nan')
        }
    
    except Exception as e:
        print(f"Error processing PDF file '{filename}': {e}")
        return None

In [ ]:
#Cell 8: define PDF processing
def process_pdf_file(file_path, filename):
    if filename.startswith("PtSrc_"):
        return process_ptsrc_pdf(file_path, filename)
    else:
        return process_regular_pdf(file_path, filename)

In [ ]:
#Cell 9: Extract PDF data from folder
def extract_pdf_data(folder_path):
    combined_data = []
    for filename in os.listdir(folder_path):
        # Check for PDF extension (case-insensitive)
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            data = process_pdf_file(file_path, filename)
            if data is not None:
                combined_data.append(data)
    return combined_data

In [ ]:
#Cell 10: Sort and process PDFs
def sort_pdf_data(combined_data, parse_numbers=False):
    combined_df = pd.DataFrame(combined_data)
    
    def extract_numeric_suffix(file_name):
        try:
            parts = file_name.split('_')[-1].split('.')[0]
            return int(parts)
        except ValueError:
            return float('nan')
    
    combined_df['File_order'] = combined_df['File'].apply(extract_numeric_suffix)
    combined_df = combined_df.sort_values(by='File_order').drop(columns=['File_order'])
    
    if parse_numbers:
        for col in ['Pb-210', 'Bi-214', 'Pb-214']:
            if col in combined_df.columns:
                combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    return combined_df

In [ ]:
#Cell 11: Export to csv
def extract_pdf_values(folder_path, output_csv_path, parse_numbers=False):
    combined_data = extract_pdf_data(folder_path)
    combined_df = sort_pdf_data(combined_data, parse_numbers)

    # Use the helper to match by depth and insert H/I/J (ptsrc, ptsrc_error, file ptsrc)
    final_df = reshape_canberra_with_ptsrc_mixed(combined_df, write_to=output_csv_path)

    csv_path = Path(output_csv_path)
    print(f"CSV saved -> {csv_path}")
    write_readme(csv_path, readme_canberra)

In [ ]:
#Cell 13: Execute functions
extract_pdf_values(folder_path, output_csv_path)

In [ ]:
#Cell 14
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Use Canberra CSV's folder as output directory
out_dir = Path(csv2_path).parent

# Auto-generate Age Model output name
output_file_name = compose_output_name("AgeModel", ".csv", out_dir)
print(f"Output file will be saved as -> {output_file_name}")


In [ ]:
#Cell 15: Load and merge data
# Load the CSV files
csv1 = pd.read_csv(csv1_path)
csv2 = pd.read_csv(csv2_path)

# Extract 'Center point of interval' from csv2 based on the median of the last digits in 'File'
csv2['Center point of interval'] = csv2['File'].apply(
    lambda x: np.median([int(num) for num in re.findall(r'\d+', x.split('_')[-1])])
)

# Merge CSV files on 'Center point of interval'
data = pd.merge(csv1, csv2, on='Center point of interval', how='left')

In [ ]:
# DEBUG: Check merge results
print(f"csv1 rows: {len(csv1)}, csv2 rows: {len(csv2)}, merged rows: {len(data)}")
print(f"csv1 center points: {sorted(csv1['Center point of interval'].unique())}")
print(f"csv2 center points: {sorted(csv2['Center point of interval'].unique())}")

In [ ]:
#Cell 17: Calculate activity and correction factors
#Update point source runs every 6 months here
#Last updated 24 November 2025
data['Pb-210 activity Uncertainty (Bq-g)'] = data['Pb-210 error']/data['sediment weight (g)']
data['Pb-210 activity (Bq/g)'] = data['Pb-210'] / data['sediment weight (g)']
data['Pb-210 correction factor'] = data['ptsrc_pb210'] / 128902.50
data['Self absorb. Corrected Pb-210 activity (Bq/g)'] = (
    data['Pb-210 activity (Bq/g)'] / data['Pb-210 correction factor']
)
data['Bi-214 activity (Bq/g)'] = data['Bi-214'] / data['sediment weight (g)']
data['Pb-214 activity (Bq/g)'] = data['Pb-214'] / data['sediment weight (g)']

In [ ]:
#Cell 18: Calculate averaged supported activity of Bi-214 and Pb-214 (Bq/g)
data['Averaged supported activity of Bi-214 and Pb-214 (Bq/g)'] = (
    data['Bi-214 activity (Bq/g)'] + data['Pb-214 activity (Bq/g)']
) / 2

# Calculate background activity uncertainty (Bq/g)
data['Background activity uncertainty (Bq/g)'] = (
    (data['Bi-214 error'] + data['Pb-214 error']) / 2
) / data['sediment weight (g)']

In [ ]:
#Cell 19: Calculate Excess Pb-210 (Bq/g)
data['Excess Pb-210 (Bq/g)'] = (
    data['Self absorb. Corrected Pb-210 activity (Bq/g)'] -
    data['Averaged supported activity of Bi-214 and Pb-214 (Bq/g)']
)

# Determine surface activity (first interval value)
data['Surface activity'] = data['Excess Pb-210 (Bq/g)'].iloc[0]

# Calculate Age bp using the natural logarithm
data['Age bp'] = (1 / 0.03114) * np.log(data['Surface activity'] / data['Excess Pb-210 (Bq/g)'])


In [ ]:
#Cell 20
def save_dataframe(df, basename: str, directory=Path.cwd(), index=False):
    """
    Saves a pandas DataFrame as CSV with _CoreNameYYYYMMDD suffix and README.
    """
    out_path = compose_output_name(basename, ".csv", directory)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=index)

    print(f"CSV saved -> {out_path}")
    write_readme(out_path, readme_agemodel)
    return out_path

In [ ]:
#Cell 21
from pathlib import Path

def _resolve_csv_target(output_csv_path, default_basename="pdf_extraction"):
    """
    Accepts either a directory or a file path. 
    - If a directory or a path without extension is provided, creates it if needed and
      returns <directory>/<default_basename>_CoreNameYYYYMMDD.csv using compose_output_name().
    - If a .csv file path is provided, returns it as-is (and ensures parent exists).
    """
    p = Path(output_csv_path)

    # Explicit .csv file path
    if p.suffix.lower() == ".csv":
        p.parent.mkdir(parents=True, exist_ok=True)
        return p

    # Treat as a directory (existing or to be created)
    if p.exists() and p.is_dir():
        return compose_output_name(default_basename, ".csv", p)

    if p.suffix == "":
        # looks like a folder path without extension; make it a directory
        p.mkdir(parents=True, exist_ok=True)
        return compose_output_name(default_basename, ".csv", p)

    # Fallback: force .csv extension
    p = p.with_suffix(".csv")
    p.parent.mkdir(parents=True, exist_ok=True)
    return p


In [ ]:
#Cell 22
def save_text(text: str, basename: str, directory=Path.cwd(), ext=".txt"):
    """
    Saves a string as text with _CoreNameYYYYMMDD suffix and README.
    """
    out_path = compose_output_name(basename, ext, directory)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    Path(out_path).write_text(text, encoding="utf-8")

    print(f"Text saved -> {out_path}")
    write_readme(out_path, readme_agemodel)

# Cell 23: Calculate 'calendar years pre year of core'
data['calendar years pre year of core'] = year_of_core - data['Age bp']

from pathlib import Path
out_dir = Path(output_file_name).parent if output_file_name else Path.cwd()
age_model_csv = save_dataframe(data, basename="AgeModel", directory=out_dir, index=False)

In [ ]:
#Cell 24
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# Ask for the age model file to be plotted (CSV format)
age_model_file = age_model_csv
data = pd.read_csv(age_model_file)


In [ ]:
#Cell 25: Get the core name for the plot title
core_name = CORE_NAME_RAW

if depths_to_label_input.lower() == 'all':
    depths_to_label = data['Center point of interval'].tolist()  # Label all intervals
else:
    depths_to_label = [float(depth.strip()) for depth in depths_to_label_input.split(",")]

In [ ]:
#Cell 27: Define colors for the data series and error bars
excess_pb210_color = 'black'
excess_pb210_error_color = mcolors.to_rgba(excess_pb210_color, alpha=0.3)
supported_activity_color = 'grey'
supported_activity_error_color = mcolors.to_rgba(supported_activity_color, alpha=0.3)

# Ask for the folder to save the plot PDF and create a filename
save_location = folder_path
plot_filename = f"{core_name}_Age_Model.pdf"
save_path = save_location.rstrip('/') + "/" + plot_filename

In [ ]:
#Cell 28
def save_figure(fig, basename: str, directory=Path.cwd(), ext=".png", dpi=300):
    """
    Adds rotated footer 'Created by <Operator> 210PbAgeModelScript v2.2'
    along the right axis spine in light grey, saves with suffix, and writes a README.
    """

    # Remove any prior footer
    for t in list(fig.texts):
        if getattr(t, "get_gid", lambda: None)() == "footer":
            try:
                t.remove()
            except Exception:
                pass

    # Grab the first axes (main plot)
    ax = fig.axes[0]

    # Add rotated text right on the spine
    footer = ax.text(
        1.01, 0.5, f"Created by {META['operator']} with 210PbAgeModelScript {META['version']}",
        ha="left", va="center", rotation=270, fontsize=9,
        color="lightgrey", transform=ax.transAxes
    )
    footer.set_gid("footer")

    # Save figure
    out_path = compose_output_name(basename, ext, directory)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_path, dpi=dpi, bbox_inches="tight")

    print(f"Figure saved -> {out_path}")
    write_readme(out_path, readme_plot)


In [ ]:
#Cell 29
plt.figure(figsize=(3, 5))
plt.errorbar(
    data['Pb-210 activity (Bq/g)'], data['Center point of interval'], 
    xerr=data['Pb-210 activity Uncertainty (Bq-g)'], fmt='-', color=excess_pb210_color, 
    label='Pb-210 activity (Bq/unit)', capsize=5, linewidth=1, 
    ecolor=excess_pb210_error_color
)
plt.xscale('log')
plt.xlim(0.01, 10)
# Highlight intervals with missing radioisotopes using brown spans
for y in missing_depths:
    plt.axhspan(y - 0.5, y + 0.5, alpha=0.5, color='brown', 
                label='Undetectable radioisotope' if y == missing_depths[0] else None)

plt.title(f"{core_name} 210 Pb Uncorrected Activity", fontsize=18)
plt.xlabel("Bq/g", fontsize=14)
plt.ylabel("Depth (cm)", fontsize=14)
plt.gca().invert_yaxis()  # Show depth from surface (invert y-axis)
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='lightgray')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=1)
plt.tight_layout()
fig = plt.gcf()
save_figure(fig, basename="AgeModelPlot", directory=save_location, ext=".pdf", dpi=300)
plt.show()



In [ ]:
#Cell 30 (UPDATED with connected lines for both Excess Pb-210 and Background Activity)
import matplotlib.patches as patches

plt.figure(figsize=(5, 10))

# Calculate vertical errors from top/base to center of interval
depth_errors = np.array([
    [center - top, base - center]
    for top, center, base in zip(
        data['Top of interval (cm)'],
        data['Center point of interval'],
        data['Base of interval (cm)']
    )
]).T  # shape (2, N)

# Calculate symmetric vertical errors
yerr = np.abs(data['Center point of interval'] - data['Top of interval (cm)'])

# Calculate symmetric horizontal errors
xerr = data['Pb-210 activity Uncertainty (Bq-g)']

# Draw connecting line for Excess Pb-210 data points
# Filter out NaN values to ensure continuous line
valid_mask_excess = ~data['Excess Pb-210 (Bq/g)'].isna()
plt.plot(
    data.loc[valid_mask_excess, 'Excess Pb-210 (Bq/g)'], 
    data.loc[valid_mask_excess, 'Center point of interval'],
    color=excess_pb210_color, linewidth=1, zorder=1
)

# Draw error boxes for Excess Pb-210
for i in range(len(data)):
    x = data['Excess Pb-210 (Bq/g)'].iloc[i]
    y = data['Center point of interval'].iloc[i]
    
    # Skip if x is NaN
    if pd.isna(x):
        continue
    
    width = xerr.iloc[i] * 2
    height = yerr.iloc[i] * 2
    rect = patches.Rectangle(
        (x - xerr.iloc[i], y - yerr.iloc[i]), width, height,
        linewidth=0.5, edgecolor='grey', facecolor='none'
    )
    plt.gca().add_patch(rect)

# Plot background activity with connecting line and horizontal error bars
# Filter out NaN values for the background activity line
valid_mask_bg = ~data['Averaged supported activity of Bi-214 and Pb-214 (Bq/g)'].isna()
plt.errorbar(
    data.loc[valid_mask_bg, 'Averaged supported activity of Bi-214 and Pb-214 (Bq/g)'],
    data.loc[valid_mask_bg, 'Center point of interval'],
    xerr=data.loc[valid_mask_bg, 'Background activity uncertainty (Bq/g)'],
    fmt='-', color=supported_activity_color, label='Background Activity',
    capsize=5, linewidth=1, ecolor=supported_activity_error_color
)

# Highlight missing intervals with brown spans
for y in missing_depths:
    plt.axhspan(y - 0.5, y + 0.5, alpha=0.5, color='brown',
                label='Undetectable radioisotope' if y == missing_depths[0] else None)

# Annotate the selected depths with rounded-up calendar years
for i, depth in enumerate(data['Center point of interval']):
    if depth in depths_to_label:
        year_value = data['calendar years pre year of core'].iloc[i]
        if not pd.isna(year_value):
            plt.text(
                data['Excess Pb-210 (Bq/g)'].iloc[i] + 0.05, depth,
                f'{int(np.ceil(year_value))}',
                fontsize=14, color='black', verticalalignment='center'
            )

plt.title(f"{core_name} Age Model", fontsize=18)
plt.xlabel("Bq/unit", fontsize=14)
plt.ylabel("Depth (cm)", fontsize=14)
plt.xscale('log')
plt.xlim(0.01, 2)
plt.gca().invert_yaxis()
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='lightgray')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=1)
plt.tight_layout()
fig = plt.gcf()
save_figure(fig, basename="AgeModelPlot", directory=save_location, ext=".pdf", dpi=300)
plt.show()

# Well done!

#### When you've finished, go to Cell > All Output > Clear to be ready for the next user of this script.